In [2]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0,
    
    model_name="llama-3.3-70b-versatile",
)
response = llm.invoke("What composed the Mona Lisa?")
print(response.content)

The Mona Lisa, a famous painting by Leonardo da Vinci, is composed of several key elements:

1. **Support**: The painting is done on a single piece of poplar wood, which was a common support material for paintings during the Renaissance.
2. **Ground**: The wood panel was prepared with a layer of gesso, a mixture of water, glue, and whiting (calcium carbonate), to create a smooth surface for painting.
3. **Paint**: The painting is executed in oil paint, with a range of pigments including:
	* Lead white (for highlights and skin tones)
	* Ultramarine blue (for the sky and clothing)
	* Vermilion (for the subject's lips and clothing)
	* Earth oxides (for the subject's skin and hair)
	* Carbon black (for shading and outlines)
4. **Techniques**: Da Vinci employed several techniques to achieve the painting's distinctive effects, including:
	* Sfumato: a method of blending colors to create a soft, hazy effect
	* Chiaroscuro: the use of strong contrasts between light and dark to create a sense o

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://candidate.aurion.cloud/acu/production/career/vacancies/2912654498620499~1/edit")
page_data = loader.load().pop().page_content
print(page_data)






















Senior Lecturer (Midwifery) - Vacancy Details – Aurion
 









            Sign in
            










      Senior Lecturer (Midwifery) - Vacancy Details
    

 





        Vacancy Details
      








    Position:
  

 




    Organisation Unit:
  

 




    Location:
  

 




    Tenure:
  

 




    Closing:
  

 




    Synopsis:
  


Campus Location: Brisbane, Australia
Job No: APTAV100150#022

Are you an experienced academic ready to make outstanding and original contributions to learning and teaching in the discipline of Midwifery?
ACU offers 17% employer superannuation, salary packaging and generous leave provisions
Full time, continuing role at our Brisbane campus with some hybrid flexibility

About the role:
Operating within Midwifery (Brisbane), the Senior Lecturer provides academic leadership in the School by undertaking coordination responsibilities and/or other leadership portfolios as required. This includes making a significant cont

In [9]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
